<a href="https://colab.research.google.com/github/amilyk/Agentic-ai-notes/blob/main/component_level_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "docstring-parser>=0.15,<1.0"
!pip install "httpx[http2]==0.28.1"
!pip install aisuite --no-deps

In [2]:
!pip install tavily
!pip install wikipedia
!pip install anthropic

组件级评估：无需评估整条流水线就能测试 AI 系统的单个部分

In [3]:
!touch .env
!echo "DEEPSEEK_API_KEY='your-deepseek-api-key'" > .env
!echo "OPENAI_API_KEY='your-openai-api-key'" >> .env
!echo "TAVILY_API_KEY='your-tavily-api-key'" >> .env
# !echo "DLAI_TAVILY_BASE_URL='https://api.tavily.com'" >> .env

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [5]:
# =========================
# 导入
# =========================

# --- 标准库
from datetime import datetime
import json
import re

# --- 第三方 ---
import aisuite as ai

# --- 本地 / 项目 ---
import research_tools
import utils

client = ai.Client()

# 调用搜索工具，查询论文

In [14]:
def find_references(task: str, model: str = "deepseek:deepseek-chat", return_messages: bool = False):
    """使用外部工具（arxiv、tavily、wikipedia）执行研究任务。"""

    prompt = f"""
    你是一个研究函数，可以访问：
    - arxiv_tool：学术论文
    - tavily_tool：通用网页搜索（需要时返回 JSON）
    - wikipedia_tool：百科式摘要

    任务：
    {task}

    今天是 {datetime.now().strftime('%Y-%m-%d')}。

    返回的结果必须包含论文或网页链接，链接内容要以https开头，不要pdf链接，相同论文链接要去重
    """.strip()

    messages = [{"role": "user", "content": prompt}]
    tools = [
        research_tools.arxiv_search_tool,
        research_tools.tavily_search_tool,
        research_tools.wikipedia_search_tool,
        # arxiv_search_tool
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_turns=5,
        )
        content = response.choices[0].message.content
        return (content, messages) if return_messages else content
    except Exception as e:
        return f"[模型错误: {e}]"

In [15]:
research_task = "查找 2 篇关于LLM最新进展的近期论文"
# research_task = "查找 2 篇关于黑洞科学最新进展的近期论文"
research_result = find_references(research_task)

utils.print_html(
    research_result,
    title="研究函数输出"
)

# 评估搜索结果
-- 论文链接来自首选域名比例

In [16]:
# Tavily 结果的首选域名列表
TOP_DOMAINS = {
    # 通用参考 / 机构 / 出版方
    "wikipedia.org", "nature.com", "science.org", "sciencemag.org", "cell.com",
    "mit.edu", "stanford.edu", "harvard.edu", "nasa.gov", "noaa.gov", "europa.eu",

    # 计算机科学 / 人工智能领域会议与索引
    "arxiv.org", "acm.org", "ieee.org", "neurips.cc", "icml.cc", "openreview.net",

    # 其他权威出版源
    "elifesciences.org", "pnas.org", "jmlr.org", "springer.com", "sciencedirect.com",

    # 额外域名（针对特定情况）
    "pbs.org", "nova.edu", "nvcc.edu", "cccco.edu",

    # 知名编程网站
    "codecademy.com", "datacamp.com"
}

def evaluate_tavily_results(TOP_DOMAINS, raw: str, min_ratio=0.4):
    """
    评估纯文本研究结果是否主要来自首选域名。

    Args:
        TOP_DOMAINS (set[str]): 首选域名集合（例如 'arxiv.org', 'nature.com'）。
        raw (str): 包含 URL 的纯文本或 Markdown。
        min_ratio (float): 通过评估所需的最小首选比例（例如 0.4 = 40%）。

    Returns:
        tuple[bool, str]: (flag, markdown_report)
            flag -> True 表示通过，False 表示未通过
            markdown_report -> Markdown 格式的评估摘要
    """

    # 从文本中提取 URL
    url_pattern = re.compile(r'https?://[^\s\]\)>\}]+', flags=re.IGNORECASE)
    urls = url_pattern.findall(raw)
    # print(urls)

    if not urls:
        return False, """### Evaluation — Tavily Preferred Domains
未检测到任何 URL。
请在研究结果中包含链接。
"""

    # 统计首选域名与总数
    total = len(urls)
    preferred_count = 0
    details = []

    for url in urls:
        domain = url.split("/")[2]
        preferred = any(td in domain for td in TOP_DOMAINS)
        if preferred:
            preferred_count += 1
        details.append(f"- {url} → {'✅ 首选域名' if preferred else '❌ 非首选域名'}")

    ratio = preferred_count / total if total > 0 else 0.0
    flag = ratio >= min_ratio

    # 生成 Markdown 格式的评估报告
    report = f"""
### 评估结果 — Tavily 首选域名
- 总链接数: {total}
- 首选链接数: {preferred_count}
- 比例: {ratio:.2%}
- 阈值: {min_ratio:.0%}
- 状态: {"✅ 通过" if flag else "❌ 未通过"}

**详细结果：**
{chr(10).join(details)}
"""
    return flag, report

In [17]:
utils.print_html(json.dumps(list(TOP_DOMAINS)[:4], indent=2), title="优选域名示例")

utils.print_html("<h3>研究结果</h3>" + research_result, title="研究结果")

flag, report = evaluate_tavily_results(TOP_DOMAINS, research_result)
utils.print_html("<pre>" + report + "</pre>", title="<h3>评估摘要</h3>")

# 自定义主题搜索
--- 评估结果

In [18]:
# === 5.1. 自主尝试：主题、比例与优选域名 ===
# 在运行单元前编辑这些参数

# topic = "量子力学的最新进展"   # <- 在此修改主题
topic = "保险行业的最新进展"
min_ratio = 0.4                   # <- 修改阈值（0.0–1.0）
run_reflection = True             # <- 设为 False 跳过步骤 4

# 简短的优选域名列表（按需编辑或扩展）
TOP_DOMAINS = {
    "wikipedia.org", "nature.com", "science.org", "arxiv.org",
    "nasa.gov", "mit.edu", "stanford.edu", "harvard.edu"
}

# 显示优选域名示例
import json
utils.print_html(
    json.dumps(sorted(list(TOP_DOMAINS)), indent=2),
    title="<h3>优选域名示例</h3>"
)

# 1) 研究
research_task = f"查找 {topic} 的 2–3 篇关键论文与可靠综述。"
research_output = find_references(research_task)
utils.print_html(research_output, title=f"<h3>{topic} 的研究结果</h3>")

# 2) 评估来源（优选域名比例）
flag, eval_md = evaluate_tavily_results(TOP_DOMAINS, research_output, min_ratio=min_ratio)
utils.print_html("<pre>" + eval_md + "</pre>", title="<h3>评估摘要</h3>")